In [1]:
import os
os.chdir("/Users/jacksonwalters/Documents/GitHub/enefit-kaggle/predict-energy-behavior-of-prosumers/")
!ls

client.csv                            gas_prices.csv
county_id_to_name_map.json            historical_weather.csv
electricity_prices.csv                public_timeseries_testing_util.py
enefit                                train.csv
example_test_files                    weather_station_to_county_mapping.csv
forecast_weather.csv


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from load_data import merged_df

In [3]:
df = merged_df()

loading train data...
loading gas_prices...
loading electricity_prices...
loading forecast_weather...
merging train and gas_prices...
merging electricity_prices...
merging forecast_weather...


In [4]:
#drop the target column and data_block_id as they are not present in test data
X = df.drop(['target'], axis=1)

In [5]:
#set the target variable
y = df['target']

In [6]:
#verify there are no NaN's
np.any(np.isnan(y))

False

In [7]:
# load the modeling modules from sklearn
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint
from sklearn.metrics import mean_absolute_error

In [8]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [9]:
#create a multivariable linear regression
regr = linear_model.LinearRegression()
regr.fit(X_train, y_train)
regr.feature_names = list(X_train.columns.values)
#predict on the test data
lin_pred = regr.predict(X_test)
print(mean_absolute_error(lin_pred,y_test))

In [ ]:
#train the random forest regressor
rf = RandomForestRegressor(n_estimators=5)
rf.fit(X_train, y_train)
rf.feature_names = list(X_train.columns.values)
#predict the target values using the random forest regressor
rf_pred = rf.predict(X_test)
#mean absolute error for random regressor
print(mean_absolute_error(y_test,rf_pred))

In [10]:
hgbr = HistGradientBoostingRegressor()
#fite the model
hgbr.fit(X_train, y_train)
hgbr.feature_names = list(X_train.columns.values)
X_train.dtypes.to_csv('../models/gradient_boost_dtypes.csv')
#predict the target values using the random forest regressor
hgbr_pred = hgbr.predict(X_test)
#mean absolute error for random regressor
print(mean_absolute_error(y_test,hgbr_pred))

29.357140193214846


In [33]:
#perform k-fold cross-validation
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)

for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    #train the model
    rf_regr = RandomForestRegressor(n_estimators=1)
    rf_regr.fit(X_train, y_train)
    #predict on the test data
    rf_pred = rf_regr.predict(X_test)
    #evaluate the model
    print(mean_absolute_error(rf_pred,y_test))

68.74932818398159
44.79244121989514
43.59930557934766
50.11175003921031
51.49235091762749


In [40]:
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
# Initialize TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=5)
# Perform time series split cross-validation
cross_val_scores = cross_val_score(rf, X, y, cv=tscv, scoring='neg_mean_absolute_error')

In [41]:
cross_val_scores

array([-50.24322373, -61.66098868, -36.89642564, -62.35650541,
       -55.67914648])

In [11]:
import pickle
# save the linear model to disk
pickle.dump(regr, open('../models/linear_model.sav', 'wb'))

NameError: name 'rf' is not defined

In [12]:
pickle.dump(hgbr, open('../models/gradient_boost_model.sav', 'wb'))

In [ ]:
pickle.dump(rf, open('../models/random_forest_model.sav', 'wb'))

In [52]:
# load the model from disk
filename = '../models/gradient_boost_model.sav'
gb_model = pickle.load(open(filename, 'rb'))